In [1]:
from sqlalchemy import create_engine, text
from bs4 import BeautifulSoup as bs
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import requests
import pymssql
import locale
import glob
import os

warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)
pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [2]:
# SQL-скрипт ниже выгружает данные по остаткам с БД Staging, доступ к ней выдаётся по учётной записи Windows по согласованию

conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада],
[Артикул поставщика], [Остаток], [Название товара], [Запрет скидки], [Скидка на изделии],
[Чистый вес], [Цена Розн., за шт], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [76]:
df = stock

In [77]:
ids = df.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()

In [78]:
ids = ids.query('Остаток >= 50')

In [80]:
idss = df[['Товарное направление', 'Товарная группа', 'Проба',
    'Артикул поставщика', 'Внутренний артикул', 'ID Сайта', 'Остаток',
    'Название товара', 'Запрет скидки', 'Скидка на изделии', 'Чистый вес',
    'Товарный кластер', 'Цена Розн., за шт', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен']]

In [81]:
ids = idss.merge(ids, how='left', on='Артикул поставщика')

In [82]:
ids = ids.drop_duplicates(subset=['Артикул поставщика'])

In [83]:
ids = ids.query('Остаток_y >= 50')

In [84]:
ids = ids[~ids['Товарное направление'].str.contains('ФУТЛЯРЫ|БУ')]
ids = ids[ids['Проба'].str.contains('585|925')]

In [85]:
ids['Товарное направление'].unique().tolist()

['СИ', 'ИФ', 'ПДК', 'ЦБ', 'БК', 'ДК', 'ВЫРИЦА', 'ЧАСЫ']

In [86]:
ids['Проба'].value_counts()

Проба
585    11603
925     8689
Name: count, dtype: int64

In [87]:
stock['Товарное направление'].value_counts(dropna=False)

Товарное направление
СИ        2764257
ИФ         907749
БК         843447
ЦБ         650125
ПДК        599475
ДК         566487
ВЫРИЦА      83971
ЧАСЫ        77040
Name: count, dtype: int64

In [88]:
sales_may1 = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-05-01_2024-05-09.xlsx')
sales_may2 = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-05-10_2024-05-19.xlsx')
sales_may3 = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга13.xlsx')

In [89]:
sales_may3['Артикул поставщика'] = sales_may3['Артикул поставщика'].astype(str)

In [90]:
sales_may = pd.concat([sales_may1, sales_may2, sales_may3], ignore_index=True)

In [91]:
articles = ids['Артикул поставщика'].tolist()
arts = ids['ID Сайта'].tolist()

In [92]:
sales_may = sales_may.dropna(subset=['Количество', 'Артикул поставщика'])

In [93]:
sales_may['Количество'].value_counts(dropna=False)

Количество
1.00     724985
-1.00      3503
Name: count, dtype: int64

In [94]:
sales_may = sales_may.groupby(by = 'Артикул поставщика', as_index=False)[['Количество']].sum()

In [95]:
sales_may.columns = ['article', 'count']

In [96]:
sales_may = sales_may.query('article in @articles')

In [97]:
sales_may.columns = ['Артикул поставщика', 'Количество']
sales_may

,Артикул поставщика,Количество
6,0-017,56.00
11,00-03-0858,7.00
13,00008,9.00
16,000094-0#ХХ,3.00
19,00016Ч,7.00
...,...,...
41708,с300-1156НЕФ,10.00
41709,с35-04,37.00
41710,с462-20,2.00
41711,с9-16,35.00


In [98]:
ids = ids.merge(sales_may, how='left', on='Артикул поставщика')

In [99]:
ids['Количество'].value_counts()

Количество
1.00      3001
2.00      2423
3.00      1742
4.00      1379
5.00       993
          ... 
229.00       1
894.00       1
223.00       1
218.00       1
262.00       1
Name: count, Length: 331, dtype: int64

In [100]:
ids.to_excel('result.xlsx', index=False)

In [101]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [102]:
images = images.drop_duplicates(subset = ['article'])

In [103]:
images['article'] = images['article'].astype(str)

In [104]:
images = images.query('article in @arts')

In [105]:
images

,article,sort,published,Ссылка на изображение,Ссылка на товар
2,1000091,3,True,https://static2.585.cloud/media/products/f2d30...,https://www.585zolotoy.ru/catalog/products/100...
5,1000181,1,True,https://static2.585.cloud/media/products/38b3b...,https://www.585zolotoy.ru/catalog/products/100...
6,1000188,3,True,https://static2.585.cloud/media/products/f3bf4...,https://www.585zolotoy.ru/catalog/products/100...
16,1000339,1,True,https://static2.585.cloud/media/products/b141d...,https://www.585zolotoy.ru/catalog/products/100...
19,1000477,1,True,https://static2.585.cloud/media/products/b1000...,https://www.585zolotoy.ru/catalog/products/100...
...,...,...,...,...,...
181626,8380091,5,True,https://static2.585.cloud/media/products/d384a...,https://www.585zolotoy.ru/catalog/products/838...
181637,8380097,5,True,https://static2.585.cloud/media/products/a3245...,https://www.585zolotoy.ru/catalog/products/838...
181642,8380099,4,True,https://static2.585.cloud/media/products/ffa1a...,https://www.585zolotoy.ru/catalog/products/838...
181646,8380101,4,True,https://static2.585.cloud/media/products/1c202...,https://www.585zolotoy.ru/catalog/products/838...


In [106]:
images = images[['article', 'Ссылка на изображение', 'published']]

In [107]:
ids

,Товарное направление,Товарная группа,Проба,Артикул поставщика,Внутренний артикул,ID Сайта,Остаток_x,Название товара,Запрет скидки,Скидка на изделии,Чистый вес,Товарный кластер,"Цена Розн., за шт",Тип изделия 1,Тип изделия 2,Тип изделия 3,Дизайн,Бренд,Гендерный признак,Ценовая корзина,Группа цен,Остаток_y,Количество
0,СИ,СИ ЦЕПЬ,925,40-53050-22,40-53050-22_СИ,1805294,1.00,Цепь Сингапур с а/г,Нет,40.00,6.58,6149_СИ,3315.00,ЦЕПЬ,БЕЗ КАМНЕЙ,БЕЛЫЙ,СИ ЦБ СИНГ,Значение по умолчанию,женские,СРЕДНЯЯ,СИ_ЦБ_СРЕДН,1363.00,130.00
1,СИ,СИ ЦЕПЬ,925,40-55050-22,40-55050-22_СИ,1445333,1.00,Цепь Ромб двойной с а/г,Нет,40.00,5.03,6122_СИ,3315.00,ЦЕПЬ,БЕЗ КАМНЕЙ,БЕЛЫЙ,СИ ЦБ РОМБ ДВОЙНОЙ,Значение по умолчанию,унисекс,СРЕДНЯЯ,СИ_ЦБ_СРЕДН,462.00,36.00
2,СИ,СИ ЦЕПЬ,925,40-55060-22,40-55060-22_СИ,1879000,1.00,Цепь Ромб двойной с а/г,Нет,40.00,8.04,6122_СИ,4482.00,ЦЕПЬ,БЕЗ КАМНЕЙ,БЕЛЫЙ,СИ ЦБ РОМБ ДВОЙНОЙ,Значение по умолчанию,унисекс,СРЕДНЯЯ,СИ_ЦБ_СРЕДН,678.00,36.00
3,СИ,СИ ЦЕПЬ,925,40-56040-22,40-56040-22_СИ,1600277,1.00,Цепь Ромб тройной с а/г,Нет,40.00,4.38,6118_СИ,3482.00,ЦЕПЬ,БЕЗ КАМНЕЙ,БЕЛЫЙ,СИ ЦБ РОМБ ТРОЙНОЙ,Значение по умолчанию,унисекс,ДЕШЕВАЯ,СИ_ЦБ_СРЕДН,955.00,61.00
4,СИ,СИ ЦЕПЬ,925,40-58035-22,40-58035-22_СИ,1793276,1.00,Цепь Якорь квадртный с а/г,Нет,40.00,1.44,6145_СИ,982.00,ЦЕПЬ,БЕЗ КАМНЕЙ,БЕЛЫЙ,СИ ЦБ ЯКОРЬ,Значение по умолчанию,унисекс,ЛЕГКОВЕС,СИ_ЦБ_СРЕДН,1210.00,81.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20287,ДК,ДК КОЛЬЦА,585,20-33-1000-13349,20-33-1000-13349_ДК,,1.00,"Кольцо с бриллиантами+шайба+алм.грань, с родир.",Нет,0.00,1.44,ДК_2020,NaN,КОЛЬЦО,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК ГЕОМЕТРИЯ,Значение по умолчанию,женские,СРЕДНЯЯ,ДК_СРЕДНЯЯ,50.00,NaN
20288,ПДК,ПДК СЕРЬГИ,585,л2166404ТЛч,л2166404ТЛч_ПДК,,1.00,"Серьги с топазом london, черн. фианитами и чер...",Нет,0.00,3.64,ДК_2542,NaN,СЕРЬГИ,С ПОЛУДРАГОЦЕННЫМИ КАМНЯМИ,ЖЕЛТЫЙ,ДК ГЕОМЕТРИЯ,Значение по умолчанию,женские,ДОРОГАЯ,ПДК_ДОРОГАЯ,69.00,NaN
20289,ИФ,ИФ ПОДВЕС ДЕКОР,585,34533,34533_ИФ,,1.00,"Подвес, фианит",Нет,0.00,0.90,ИФ_1095,NaN,ПОДВЕС ДЕКОРАТИВНЫЙ,С ФИАНИТАМИ,КРАСНЫЙ,ИФ БУКВЫ,SOKOLOV,женские,СРЕДНЯЯ,ИФ.БК_СР,50.00,NaN
20290,СИ,СИ ПОДВЕС БК,925,50-76-0000-15294,50-76-0000-15294_СИ,,1.00,"Подвеска род., черн. лк",Нет,40.00,3.45,6401_СИ,4487.00,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,ЧЁРНЫЙ,СИ ПРОЧЕЕ,Значение по умолчанию,унисекс,СРЕДНЯЯ,СИ_ПОДВЕС_ДЕКОР_СРЕДН,50.00,NaN


In [108]:
images.columns = ['ID Сайта', 'Ссылка на изображение', 'published']

In [109]:
ids = ids.merge(images, how='left', on='ID Сайта')

In [110]:
ids.columns

Index(['Товарное направление', 'Товарная группа', 'Проба',
       'Артикул поставщика', 'Внутренний артикул', 'ID Сайта', 'Остаток_x',
       'Название товара', 'Запрет скидки', 'Скидка на изделии', 'Чистый вес',
       'Товарный кластер', 'Цена Розн., за шт', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток_y',
       'Количество', 'Ссылка на изображение', 'published'],
      dtype='object')

In [111]:
ids.columns = ['Товарное направление', 'Товарная группа', 'Проба',
'Артикул поставщика', 'Внутренний артикул', 'ID Сайта', 'Остаток_x',
'Название товара', 'Запрет скидки', 'Скидка на изделии', 'Чистый вес',
'Товарный кластер', 'Цена Розн., за шт', 'Тип изделия 1',
'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток_y',
'Количество продаж за май', 'Ссылка на изображение', 'published']

In [112]:
ids['Товарная группа'].unique().tolist()

['СИ ЦЕПЬ',
 'СИ СЕРЬГИ РАЗН',
 'СИ ПОДВЕС\xa0ЦВ/К',
 'СИ КОЛЬЦО ПЕЧАТ',
 'СИ КОЛЬЦО ИФ',
 'СИ ПОДВЕС ИФ',
 'СИ КОЛЬЦО БК',
 'СИ БРАСЛЕТ СК',
 'СИ БРАСЛЕТ БК Декор',
 'СИ БРАСЛЕТ БК',
 'СИ СЕРЬГИ ИФ',
 'СИ ПОДВЕС БК',
 'СИ СЕРЬГИ\xa0 ЦВ/К',
 'СИ ШАРМЫ',
 'ИФ КОЛЬЦА',
 'ИФ СЕРЬГИ',
 'ИФ ПОДВЕС ДЕКОР',
 'ПДК СЕРЬГИ',
 'СИ СЕРЬГИ БК',
 'ПДК КОЛЬЦА',
 'ЦБ ЦЕПИ',
 'ИФ ПЕЧАТКИ',
 'СИ КОЛЬЦО ЦВ/К',
 'СИ ЖЕМЧУГ',
 'БК КОЛЬЦА ОБРУЧ',
 'ЦБ БРАСЛЕТЫ',
 'СИ БРАСЛЕТ ЦВ/К',
 'ИФ ПОДВЕС КУЛЬТ',
 'ДК КОЛЬЦА',
 'БК КОЛЬЦА',
 'БК СЕРЬГИ',
 'ДК ПОДВЕСКИ',
 'ИФ КОЛЬЦА ОБРУЧ',
 'СИ ОБРУЧАЛЬНОЕ',
 'ДК КОЛЬЦА ОБРУЧ',
 'ВЫРИЦА СЕРЕБРО',
 'СИ КОЛЬЕ СК',
 'ДК СЕРЬГИ',
 'БК ПОДВЕС КУЛЬТ',
 'ПДК ПОДВЕСКИ',
 'ДК ПЕЧАТКИ',
 'СИ АКСЕССУАР',
 'ОПТ ИФ',
 'БК ПОДВЕС ДЕКОР',
 'СИ КОЛЬЕ БК',
 'СИ КОЛЬЕ С ЭМАЛЬЮ',
 'ПДК ПЕЧАТКИ',
 'СИ КЕРАМИКА',
 'СИ ЭМАЛЬ',
 'СИ КОЛЬЕ ЦВ/К',
 'ЧАСЫ',
 'СИ ПОДАРКИ КРОСС',
 'ОПТ ОБРУЧИ',
 'ОПТ БК',
 'БК ПЕЧАТКИ',
 'СИ ПОСУДА',
 'ОПТ ПДК',
 'ОПТ КОНГО',
 'СИ ЯНТАРЬ',
 'ВЫРИЦА ЗОЛОТО',
 'БК 

In [113]:
ids = ids[~ids['Товарная группа'].str.contains('АКСЕССУАР|ОПТ|ПОСУДА|КРОСС|ЦВ/К|РАЗН|КЕРАМИКА|ЭМАЛЬ|ЯНТАРЬ')]

In [114]:
ids = ids.drop(columns=['Остаток_x'])

In [115]:
ids['price_personal'] = np.where(
    ids['Запрет скидки'] == 'Да', ids['Цена Розн., за шт'], ids['Цена Розн., за шт'] * 0.6 * 0.8 * (1 - (ids['Скидка на изделии'] / 100))
)

In [124]:
ids = ids.drop_duplicates(subset=['Товарное направление', 'Товарная группа', 'Проба',
    'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Название товара', 'Запрет скидки', 'Скидка на изделии', 'Чистый вес',
    'Товарный кластер', 'Цена Розн., за шт', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток_y',
    'Количество продаж за май', 'Ссылка на изображение', 'published',
    'price_personal'])

In [125]:
ids.to_excel('result.xlsx', index=False)

In [126]:
ids.columns

Index(['Товарное направление', 'Товарная группа', 'Проба',
       'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
       'Название товара', 'Запрет скидки', 'Скидка на изделии', 'Чистый вес',
       'Товарный кластер', 'Цена Розн., за шт', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток_y',
       'Количество продаж за май', 'Ссылка на изображение', 'published',
       'price_personal', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)'],
      dtype='object')

In [127]:
ids['Наценка'] = ids['price_personal'] / ids['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [128]:
# ids = ids.dropna(subset=['published'])

In [129]:
ids.columns = ['Товарное направление', 'Товарная группа', 'Проба',
    'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Название товара', 'Запрет скидки', 'Скидка на изделии', 'Чистый вес',
    'Товарный кластер', 'Цена Розн., за шт', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
    'Количество продаж за май', 'Ссылка на изображение', 'published',
    'Итоговая цена на изделии 40%+20%', 'Себестоимость', 'Наценка']

In [130]:
ids = ids[['Товарное направление', 'Товарная группа', 'Проба',
    'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Название товара', 'Чистый вес',
    'Товарный кластер', 'Цена Розн., за шт', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
    'Количество продаж за май', 'Ссылка на изображение', 'published',
    'Итоговая цена на изделии 40%+20%', 'Себестоимость', 'Наценка']]

In [131]:
ids.to_excel('с правками.xlsx', index=False)

In [3]:
stock

,Артикул поставщика,Остаток,Название товара,Запрет скидки,Скидка на изделии,Чистый вес,"Цена Розн., за шт",Дата закупки
0,К204,1.00,"Подвес с фиан. род., поз.",Нет,40.00,7.34,9931.00,2023-02-28
1,КЛ13,1.00,Подвес с фиан. род.,Нет,40.00,8.02,11650.00,2023-02-28
2,КЛ13,1.00,Подвес с фиан. род.,Нет,40.00,8.14,11650.00,2023-02-28
3,КЛ05,1.00,Подвес с фиан. род.,Нет,40.00,4.89,7483.00,2023-02-28
4,КЛ05,1.00,Подвес с фиан. род.,Нет,40.00,5.01,7483.00,2023-02-28
...,...,...,...,...,...,...,...,...
6503254,Ср925Р-6А8800000Ф1,1.00,Подвес с фиан. род.,Нет,40.00,0.83,1482.00,2023-12-13
6503255,Ср925Р-6А9200000Ф1,1.00,Подвес с фиан. род.,Нет,40.00,1.01,1648.00,2023-12-13
6503256,Ср925Р-8А3203016Ф1,1.00,Брасле с фиан.род.,Нет,40.00,2.21,3315.00,2024-04-17
6503257,Ср925Р-8А3203040Ф1,1.00,Колье с фиан. род.,Нет,40.00,3.32,4148.00,2023-12-13


In [4]:
df = pd.read_excel('с правками 1.xlsx')

In [7]:
stock['Дата закупки'] = pd.to_datetime(stock['Дата закупки'], dayfirst=True)

In [8]:
min_date = stock.groupby(by = 'Артикул поставщика', as_index=False)['Дата закупки'].min()

In [10]:
min_date.columns = ['article', 'date']

In [12]:
articles = df['Артикул поставщика'].tolist()

In [13]:
min_date = min_date.query('article in @articles')

In [15]:
min_date.columns = ['Артикул поставщика', 'Дата закупки']

In [16]:
df = df.merge(min_date, how='left', on='Артикул поставщика')

In [18]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-11.xlsx']

In [20]:
sale = pd.DataFrame()
for name in all_filenames:
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

In [54]:
x = sale

In [55]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [56]:
sale = sale[sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')]

In [57]:
sale.columns = ['КБК', 'Дата', 'Продавец', 'Номер чека', 'ID продажи', 'Тов',
    'article', 'Количество', 'Номенклатура', 'Номер карты',
    'Товарная группа', 'Товарное направление', 'Тип 1',
    'Списание СберСпасибо', 'Группа наценки', 'Вес', 'Проба', 'Размер',
    'ШК', 'Цена без скидки', 'Базовая скидка', 'Скидка в руб.', 'Название',
    'Цена после скидки', 'Phone', 'Себестоимость', 'Сумма', 'Услуга',
    'Общая сумма']

In [58]:
sale = sale.query('article in @articles')

In [59]:
sale = sale.groupby(by = 'article', as_index=False)[['Количество', 'Общая сумма']].sum()

In [62]:
sale.columns = ['Артикул поставщика', 'Количество продаж', 'Общая сумма продаж']

In [65]:
df = df.merge(sale, how='left', on='Артикул поставщика')

In [68]:
df.columns = ['Товарное направление', 'Товарная группа', 'Проба',
    'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Название товара', 'Чистый вес', 'Товарный кластер',
    'Цена Розн., за шт', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Остаток', 'Количество продаж за май', 'Ссылка на изображение',
    'Опубликован', 'Текущая итоговая цена на изделии 40%+20%', 'Себестоимость',
    'Наценка', 'Минимальная дата закупки', 'Количество продаж 14.03-11.06', 'Общая сумма продаж 14.03-11.06']

In [116]:
# SQL-скрипт ниже выгружает данные по остаткам с БД Staging, доступ к ней выдаётся по учётной записи Windows по согласованию

conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Название склада],
[Артикул поставщика], [Остаток], [Название товара], [Запрет скидки], [Скидка на изделии],
[Чистый вес], [Цена Розн., за шт], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
ostatok = pd.DataFrame(sql_query)

In [130]:
ost = ostatok

In [132]:
ost.columns = ['Код склада', 'name', 'article', 'Остаток', 'Название товара',
    'Запрет скидки', 'Скидка на изделии', 'Чистый вес', 'Цена Розн., за шт',
    'Дата закупки']

In [134]:
ost = ost.query('article in @articles')

In [135]:
ost

,Код склада,name,article,Остаток,Название товара,Запрет скидки,Скидка на изделии,Чистый вес,"Цена Розн., за шт",Дата закупки
1,5814,"Лабинск, Красная, 25",КЛ13,1.00,Подвес с фиан. род.,Нет,40.00,8.02,11650.00,2023-02-28
2,5814,"Лабинск, Красная, 25",КЛ13,1.00,Подвес с фиан. род.,Нет,40.00,8.14,11650.00,2023-02-28
8,5814,"Лабинск, Красная, 25",КЧ27,1.00,Подвес с фиан. род.,Нет,40.00,1.69,2483.00,2023-02-28
9,5814,"Лабинск, Красная, 25",КЧ34,1.00,Подвес с фиан. род.,Нет,40.00,2.23,3017.00,2023-02-28
10,5814,"Лабинск, Красная, 25",КЧ40,1.00,Подвес с фиан. род.,Нет,40.00,4.60,5817.00,2023-02-28
...,...,...,...,...,...,...,...,...,...,...
6503253,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-6А3100000Ф1,1.00,Подвес с фиан. род.,Нет,40.00,0.26,1482.00,2023-12-13
6503254,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-6А8800000Ф1,1.00,Подвес с фиан. род.,Нет,40.00,0.83,1482.00,2023-12-13
6503255,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-6А9200000Ф1,1.00,Подвес с фиан. род.,Нет,40.00,1.01,1648.00,2023-12-13
6503256,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-8А3203016Ф1,1.00,Брасле с фиан.род.,Нет,40.00,2.21,3315.00,2024-04-17


In [136]:
ost = ost[['Код склада', 'name', 'article', 'Остаток']]

In [137]:
ost['name'] = ost['name'].astype(str)

In [140]:
center = ost[ost['Код склада'] == '3075']
center = center.groupby(by = 'article', as_index=False)['Остаток'].sum()
center.columns = ['Артикул поставщика', 'Остаток на центральном складе']

In [141]:
kolambus = ost[ost['Код склада'] == '5442']
kolambus = kolambus.groupby(by = 'article', as_index=False)['Остаток'].sum()
kolambus.columns = ['Артикул поставщика', 'Остаток на складе коламбуса']

In [142]:
federal = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Федеральные склады 11.06.xlsx')

In [143]:
federal = federal['outlet_name'].tolist() 

In [ ]:
fed = ost.query()

In [147]:
ost

,Код склада,name,article,Остаток
1,5814,"Лабинск, Красная, 25",КЛ13,1.00
2,5814,"Лабинск, Красная, 25",КЛ13,1.00
8,5814,"Лабинск, Красная, 25",КЧ27,1.00
9,5814,"Лабинск, Красная, 25",КЧ34,1.00
10,5814,"Лабинск, Красная, 25",КЧ40,1.00
...,...,...,...,...
6503253,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-6А3100000Ф1,1.00
6503254,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-6А8800000Ф1,1.00
6503255,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-6А9200000Ф1,1.00
6503256,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Ср925Р-8А3203016Ф1,1.00


In [148]:
federal = ost.query('name in @federal')

In [150]:
federal = federal.groupby(by = 'article', as_index=False)['Остаток'].sum()
federal.columns = ['Артикул поставщика', 'Остаток на федеральных складах']

In [151]:
federal

,Артикул поставщика,Остаток на федеральных складах
0,00-03-0858,1.00
1,0008.2.9.33,2.00
2,01-0075-00-710-02-01,4.00
3,01-1296/00КЦ-00,1.00
4,01-1800/0000-00,3.00
...,...,...
10663,с22цр,4.00
10664,с302-20н/к,1.00
10665,с35-04,6.00
10666,с9-16,4.00


In [153]:
df = df.merge(federal, how='left', on='Артикул поставщика')

In [154]:
df = df.merge(kolambus, how='left', on='Артикул поставщика')

In [155]:
df = df.merge(center, how='left', on='Артикул поставщика')

In [157]:
df.to_excel('предварительный.xlsx', index=False)

In [159]:
df['Сумма в себестоимости на остатках'] = df['Себестоимость'] * df['Остаток']

In [161]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [166]:
zapret.columns = ['ТГ', 'article', 'Описание', 'Рейтинг по размещению 30.04.2024',
    'Артикул сайта', 'Опубликован', 'Остаток на 30.04.24',
    'Себестоимость остатка на 30.04.2024',
    'Среднее количество продаж, шт/мес', 'Средняя сумма продаж, руб/мес',
    'Cредняя себестоимость, руб/мес', 'Средняя валовая прибыль,  руб/мес',
    'Средняя цена продажи, шт', 'Причина блокировки']

In [167]:
zapret = zapret['article'].tolist()

False

In [173]:
df['Запрет к размещению'] = np.nan
for i in df.index:
    if df['Артикул поставщика'][i] in zapret:
        df['Запрет к размещению'][i] = True
    else:
        df['Запрет к размещению'][i] = False

In [175]:
df['Запрет к размещению'].value_counts()

Запрет к размещению
False    17311
True        36
Name: count, dtype: int64

In [181]:
df['rank1'] = df['Остаток на федеральных складах'].rank()
df['rank2'] = df['Сумма в себестоимости на остатках'].rank()
df['rank3'] = df['Общая сумма продаж 14.03-11.06'].rank()

In [184]:
df['rank'] = (df['rank1'] + df['rank2'] + df['rank3'])/3

In [186]:
df = df.sort_values(by = ['rank'], ascending=False)

In [199]:
df = df.reset_index()
df['Рейтинг'] = np.nan
for i in df.index:
    df['Рейтинг'][i] = i + 1

In [201]:
df = df.drop(columns=['rank1', 'rank2', 'rank3'])

In [203]:
df

,level_0,index,Товарное направление,Товарная группа,Проба,Артикул поставщика,Внутренний артикул,ID Сайта,Название товара,Чистый вес,Товарный кластер,"Цена Розн., за шт",Тип изделия 1,Тип изделия 2,Тип изделия 3,Дизайн,Бренд,Гендерный признак,Ценовая корзина,Группа цен,Остаток,Количество продаж за май,Ссылка на изображение,Опубликован,Текущая итоговая цена на изделии 40%+20%,Себестоимость,Наценка,Минимальная дата закупки,Количество продаж 14.03-11.06,Общая сумма продаж 14.03-11.06,Остаток на федеральных складах,Остаток на складе коламбуса,Остаток на центральном складе,Сумма в себестоимости на остатках,Запрет к размещению,rank,Рейтинг
0,0,402,БК,БК КОЛЬЦА ОБРУЧ,585,КО 15-00,КО 15-00_БК,2004212.00,Кольцо обручальное,2.78,БК_1524_16,42673.00,КОЛЬЦО ОБРУЧАЛЬНОЕ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ ОБРУЧ ПЛОСКАЯ,Значение по умолчанию,унисекс,ДОРОГАЯ,ИФ.БК_ДОР,25772,2203.00,https://static2.585.cloud/media/products/28c8a...,1.00,16386.43,7949.90,1.06,2022-07-02,5488.00,120412134.66,163.00,92.00,221.00,204884822.80,False,14783.33,1.00
1,1,806,БК,БК КОЛЬЦА ОБРУЧ,585,КО 04-00,КО 04-00_БК,2002944.00,Кольцо обручальное,2.04,"БК_1519_15,5",12990.00,КОЛЬЦО ОБРУЧАЛЬНОЕ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ ОБРУЧ ГЛАДКАЯ,NaN,унисекс,СРЕДНЯЯ,KVI,17607,6536.00,https://static2.585.cloud/media/products/9c805...,1.00,12990.00,5251.70,1.47,2022-07-02,10513.00,148563223.27,146.00,80.00,840.00,92466681.90,False,14780.67,2.00
2,2,805,БК,БК КОЛЬЦА ОБРУЧ,585,КО 03-00,КО 03-00_БК,2002928.00,Кольцо обручальное,1.52,"БК_1518_15,5",30793.00,КОЛЬЦО ОБРУЧАЛЬНОЕ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ ОБРУЧ ГЛАДКАЯ,NaN,унисекс,ДЕШЕВАЯ,ИФ.БК_ДЕШ,21081,2135.00,https://static2.585.cloud/media/products/5432f...,1.00,8277.16,3961.31,1.09,2022-07-02,6400.00,81818441.62,150.00,80.00,368.00,83508376.11,False,14777.67,3.00
3,3,2085,ЦБ,ЦБ БРАСЛЕТЫ,585,431-01-0080-37654,431-01-0080-37654_ЦБ,7862577.00,Браслет Бисмарк пуст с а/г,3.65,7625_ЦБ,64569.00,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,БИСМАРК ПУСТ,ADRIATICA,женские,СРЕДНЯЯ,ЦБ_ПУСТОТЕЛЫЕ >3 ГРАММ,11043,689.00,https://static2.585.cloud/media/products/ad6f6...,1.00,24794.50,13000.67,0.91,2022-08-02,1748.00,51810596.03,70.00,86.00,458.00,143566398.81,False,14776.33,4.00
4,4,807,БК,БК КОЛЬЦА ОБРУЧ,585,КО 05-00,КО 05-00_БК,2002948.00,Кольцо обручальное,2.85,"БК_1520_16,5",43748.00,КОЛЬЦО ОБРУЧАЛЬНОЕ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ ОБРУЧ ГЛАДКАЯ,NaN,унисекс,СРЕДНЯЯ ДОРОГАЯ,ИФ.БК_СР,15686,1156.00,https://static2.585.cloud/media/products/66dfc...,1.00,16799.23,5348.96,2.14,2022-07-02,3157.00,62820816.30,128.00,67.00,237.00,83903786.56,False,14775.67,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17342,17342,17342,ДК,ДК КОЛЬЦА,585,20-33-1000-13349,20-33-1000-13349_ДК,NaN,"Кольцо с бриллиантами+шайба+алм.грань, с родир.",1.44,ДК_2020,NaN,КОЛЬЦО,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК ГЕОМЕТРИЯ,Значение по умолчанию,женские,СРЕДНЯЯ,ДК_СРЕДНЯЯ,50,NaN,NaN,NaN,NaN,7679.53,NaN,2024-06-10,NaN,NaN,NaN,NaN,99.00,383976.50,False,NaN,17343.00
17343,17343,17343,ПДК,ПДК СЕРЬГИ,585,л2166404ТЛч,л2166404ТЛч_ПДК,NaN,"Серьги с топазом london, черн. фианитами и чер...",3.64,ДК_2542,NaN,СЕРЬГИ,С ПОЛУДРАГОЦЕННЫМИ КАМНЯМИ,ЖЕЛТЫЙ,ДК ГЕОМЕТРИЯ,Значение по умолчанию,женские,ДОРОГАЯ,ПДК_ДОРОГАЯ,69,NaN,NaN,NaN,NaN,15505.14,NaN,2024-05-31,NaN,NaN,NaN,NaN,69.00,1069854.66,False,NaN,17344.00
17344,17344,17344,ИФ,ИФ ПОДВЕС ДЕКОР,585,34533,34533_ИФ,NaN,"Подвес, фианит",0.90,ИФ_1095,NaN,ПОДВЕС ДЕКОРАТИВНЫЙ,С ФИАНИТАМИ,КРАСНЫЙ,ИФ БУКВЫ,SOKOLOV,женские,СРЕДНЯЯ,ИФ.БК_СР,50,NaN,NaN,NaN,NaN,3511.99,NaN,2024-06-09,NaN,NaN,NaN,NaN,50.00,175599.50,False,NaN,17345.00
17345,17345,17345,СИ,СИ ПОДВЕС БК,925,50-76-0000-15294,50-76-0000-15294_СИ,NaN,"Подвеска род., черн. лк",3.45,6401_СИ,4487.00,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,ЧЁРНЫЙ,СИ ПРОЧЕЕ,Значение по умолчанию,унисекс,СРЕДНЯЯ,СИ_ПОДВЕС_ДЕКОР_СРЕДН,50,NaN,NaN,NaN,1292.26,768.94,0.68,2024-05-31,NaN,NaN,NaN,NaN,49.00,38447.00,False,NaN,17346.00


In [204]:
df.to_excel('final.xlsx', index=False)